In [28]:
import regex as re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait # Wait class to wait/sleep until conditions are met
from selenium.webdriver.support import expected_conditions as ec # Conditions to check in the website

In [29]:
# Set Web Url to scrape
web_url = 'https://steamcommunity.com/app/1172470/reviews/?browsefilter=toprated&snr=1_5_100010_'

In [30]:
# Open Browser and go to url
driver = webdriver.Edge()
driver.get(web_url)

Exception managing MicrosoftEdge: error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_WINDOWS)
The msedgedriver version (138.0.3351.109) detected in PATH at D:\zPersonal\Tools\Selenium WebDrivers\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (139.0.3405.125); currently, msedgedriver  is recommended for MicrosoftEdge 139.*, so it is advised to delete the driver in PATH and retry


In [31]:
reviews = driver.find_elements(By.CLASS_NAME, 'apphub_CardTextContent')
len(reviews)

10

In [32]:
print(reviews[-1].text.split(maxsplit = 4)[-1])

I've been playing this game since release, and it used to be a blast. From initial release in February 2019 to about 6-9 months after Valkyrie’s release (~ early 2022), it was incredible. But not soon after this, it began its slow, agonizing decline into the utter disaster it is today. And we all know why: it’s thanks to the absolute trainwreck of a company known as EA (or as I like to call them, Electronic Arseholes).

The matchmaking? A joke. The server and connection issues? Relentless. Cheating and exploiting have become so rampant it feels like a rigged game at this point. EA’s "brilliant" solution to these issues? Nothing. Absolutely nothing. They sit on their hands while the game crumbles beneath their feet.

The Battle Pass changes? Just another example of EA's contempt for its players. Instead of addressing the fundamental problems, they milk their loyal fanbase dry, squeezing every last cent out of them. And for what? A broken, glitch-ridden mess that barely resembles the gam

In [33]:
# Scroll down
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [34]:
reviews = driver.find_elements(By.CLASS_NAME, 'apphub_CardTextContent')
reviews = [review.text.split(maxsplit = 4)[-1] for review in reviews]
len(reviews)

10

In [35]:
# Get author
authors = driver.find_elements(By.XPATH, '//div[contains(@class, "apphub_CardContentAuthorName")]/child::a[2]')
authors = [author.text for author in authors]
len(authors)

10

In [36]:
# Get Review Comments
comments = driver.find_elements(By.XPATH, '//div[@class="apphub_CardCommentCount alignNews"]')
comments = [comment.text for comment in comments]
len(comments)

20

In [37]:
# Get number of products on author's account
auth_products = driver.find_elements(By.XPATH, '//div[@class="apphub_CardContentMoreLink ellipsis"]')
auth_products = [product.text if product.text != '' else 'Few products in account' for product in auth_products] # Assuming that people without this specific title has insufficient products
len(auth_products)

20

In [38]:
# Get number of hours on author's account
auth_hrs = driver.find_elements(By.XPATH, '//div[@class = "hours"]')
auth_hrs = [hours.text for hours in auth_hrs]
len(auth_hrs)

20

In [39]:
# Get recommendation
remarks = driver.find_elements(By.XPATH,'//div[@class="title"]')
remarks = [remark.text for remark in remarks]
len(remarks)

20

In [40]:
# Found user_reactions
reactions = driver.find_elements(By.XPATH, '//div[@class="found_helpful"]')

In [41]:
reactions[0].text

'3,796 people found this review helpful\n133 people found this review funny 178'

In [42]:
[re.findall(r'[\d,]+', i.text) for i in reactions][0]

['3,796', '133', '178']

In [43]:
'No one has rated this review as helpful yet\n1 person found this review funny 58'.index('funny')

71

In [44]:
re.findall(r'[\d,]+','No one has rated this review as helpful yet\n1 person found this review funny 58')

['1', '58']

In [45]:
int(0)

0

In [46]:
def get_num(text: str, endpos: int = None, startpos: int = 0) -> int:
    num = re.findall(r'[\d,]+', text, pos = startpos, endpos = endpos) if endpos is not None else re.findall(r'[\d,]+', text, pos = startpos)
    
    if len(num) == 0:
        num = '0'
    else:
        num = num[0]
    
    return int(re.sub(r'\D','',num))

In [47]:
helpful = [get_num(i.text, endpos = i.text.lower().index('helpful')) for i in reactions]
funny = [get_num(i.text, startpos = i.text.lower().index('helpful'), endpos = i.text.lower().index('funny') if 'funny' in i.text.lower() else 0) for i in reactions]
awards = [get_num(i.text, startpos = i.text.lower().index('funny') if 'funny' in i.text.lower() else i.text.lower().index('helpful')) for i in reactions]

In [48]:
# Wait until a screen is not visible (Its inverse is visibility_of_element_located)
WebDriverWait(driver, 10).until(ec.invisibility_of_element_located((By.XPATH,'//div[@class="apphub_GetMoreContentWait"]')))

<selenium.webdriver.remote.webelement.WebElement (session="9eae8db4f77d5efe19fa5aee6a3a414b", element="f.D1EC4BF86B3F2EBC4590CB3F423E8959.d.05B5D545BAE4614497123C9E7CB9AB79.e.15")>

In [49]:
n = 1000

while len(reviews) < n:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 5).until(ec.invisibility_of_element_located((By.XPATH,'//div[@class="apphub_GetMoreContentWait"]'))) # Wait until loading screen is not visible
    reviews = driver.find_elements(By.CLASS_NAME, 'apphub_CardTextContent')  
    authors = driver.find_elements(By.XPATH, '//div[contains(@class, "apphub_CardContentAuthorName")]/child::a[2]')    
    comments = driver.find_elements(By.XPATH, '//div[@class="apphub_CardCommentCount alignNews"]')    
    auth_products = driver.find_elements(By.XPATH, '//div[@class="apphub_CardContentMoreLink ellipsis"]')
    auth_hrs = driver.find_elements(By.XPATH, '//div[@class = "hours"]')
    remarks = driver.find_elements(By.XPATH,'//div[@class="title"]')
    reactions = driver.find_elements(By.XPATH, '//div[@class="found_helpful"]')
    

reviews = [review.text.split(maxsplit = 4)[-1] for review in reviews]
authors = [author.text for author in authors]
comments = [comment.text for comment in comments]
auth_products = [product.text if product.text != '' else 'Few products in account' for product in auth_products]
auth_hrs = [hours.text for hours in auth_hrs]
remarks = [remark.text for remark in remarks]
helpful = [get_num(i.text, endpos = i.text.lower().index('helpful')) for i in reactions]
funny = [get_num(i.text, startpos = i.text.lower().index('helpful'), endpos = i.text.lower().index('funny') if 'funny' in i.text.lower() else 0) for i in reactions]
awards = [get_num(i.text, startpos = i.text.lower().index('funny') if 'funny' in i.text.lower() else i.text.lower().index('helpful')) for i in reactions]

In [52]:
len(reviews), len(authors), len(comments), len(auth_products), len(auth_hrs), len(remarks), len(awards), len(helpful), len(funny)

(1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000)

In [53]:
# Combine Author and review using pandas
steam_reviews = pd.DataFrame({
    'Author': authors,
    'Review': reviews,
    '# of Comments':comments,
    'Products':auth_products,
    'Hours on Account':auth_hrs,
    'Remark':remarks,
    'Users found review funny': funny,
    'Users found review helpful': helpful,
    'Community Awards': awards
})

print(len(steam_reviews))
display(steam_reviews.head(10))

1000


,Author,Review,# of Comments,Products,Hours on Account,Remark,Users found review funny,Users found review helpful,Community Awards
0,Kaladin,2 10 dollar battle passes per season\n\nTitanf...,0,228 products in account,4.1 hrs on record,Not Recommended,133,3796,178
1,KEVE,"Now with the new battle pass system, I'm delet...",78,42 products in account,"1,865.6 hrs on record",Not Recommended,85,4370,186
2,⛧ M♱ropele ⛧,At least you don't have to build an apartment ...,54,655 products in account,9.6 hrs on record,Recommended,2017,3089,500
3,to infinity and beyond,Finally I can remove Origin.,0,Few products in account,1.1 hrs on record,Recommended,634,1770,103
4,chelovek bez vkusa,Finally here\n\nIt launches without Origin...\...,35,295 products in account,16.6 hrs on record,Recommended,91,2109,49
5,Shanks,uninstall origin,0,Few products in account,16.1 hrs on record,Recommended,281,1180,64
6,azure,doesnt launch with origin\n\nedit: easy anti-c...,117,195 products in account,546.0 hrs on record,Recommended,915,6142,355
7,🐾Gizmo🐾,We don't have a cheater problem.\nOk we do hav...,48,508 products in account,"2,129.8 hrs on record",Not Recommended,85,2120,211
8,Blödhgarm,"I've got nearly 3k hours in this game, most of...",0,Few products in account,779.9 hrs on record,Not Recommended,28,1487,247
9,DeeDee MegaDooDoo,"I've been playing this game since release, and...",16,Few products in account,"1,756.3 hrs on record",Not Recommended,21,485,59


In [54]:
steam_reviews.to_excel('steam reviews.xlsx')

In [55]:
driver.quit()